### Importing necesary dependencies

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Input, Dense, Dropout

from glob import glob
import os
import gc
import random
import cv2
import yaml
import numpy as np
from sklearn.utils import shuffle
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

## Getting Data

### Image paths and labels

In [ ]:
img_paths = []
labels = []

### Getting Data from simulator

In [ ]:
SIM_DATA_DIR = os.path.join('data', 'sim')

sim_file_paths_green = glob(os.path.join(SIM_DATA_DIR, 'green*', '*.png'))
sim_file_paths_yellow = glob(os.path.join(SIM_DATA_DIR, 'yellow*', '*.png'))
sim_file_paths_red = glob(os.path.join(SIM_DATA_DIR, 'red*', '*.png'))
sim_file_paths_none = glob(os.path.join(SIM_DATA_DIR, 'none*', '*.png'))
print('From Simulator - Green: {}, Yellow: {}, Red: {}, None: {}'.format(
    len(sim_file_paths_green), len(sim_file_paths_yellow), len(sim_file_paths_red), len(sim_file_paths_none)))


img_paths.extend(sim_file_paths_green)
labels.extend([[1.0, 0.0, 0.0, 0.0] for i in range(len(sim_file_paths_green))])

img_paths.extend(sim_file_paths_yellow)
labels.extend([[0.0, 1.0, 0.0, 0.0] for i in range(len(sim_file_paths_yellow))])

img_paths.extend(sim_file_paths_red)
labels.extend([[0.0, 0.0, 1.0, 0.0] for i in range(len(sim_file_paths_red))])

img_paths.extend(sim_file_paths_none)
labels.extend([[0.0, 0.0, 0.0, 1.0] for i in range(len(sim_file_paths_none))])

### Getting Data from rosbag

In [ ]:
ROSBAG_DATA_DIR = os.path.join('data', 'rosbag')

rosbag_file_paths_green = glob(os.path.join(ROSBAG_DATA_DIR, 'green*', '*.jpg'))
rosbag_file_paths_yellow = glob(os.path.join(ROSBAG_DATA_DIR, 'yellow*', '*.jpg'))
rosbag_file_paths_red = glob(os.path.join(ROSBAG_DATA_DIR, 'red*', '*.jpg'))
rosbag_file_paths_none = glob(os.path.join(ROSBAG_DATA_DIR, 'none*', '*.jpg'))
print('From Rosbag - Green: {}, Yellow: {}, Red: {}, None: {}'.format(
    len(rosbag_file_paths_green), len(rosbag_file_paths_yellow), len(rosbag_file_paths_red),
    len(rosbag_file_paths_none)))


img_paths.extend(rosbag_file_paths_green)
labels.extend([[1.0, 0.0, 0.0, 0.0] for i in range(len(rosbag_file_paths_green))])

img_paths.extend(rosbag_file_paths_yellow)
labels.extend([[0.0, 1.0, 0.0, 0.0] for i in range(len(rosbag_file_paths_yellow))])

img_paths.extend(rosbag_file_paths_red)
labels.extend([[0.0, 0.0, 1.0, 0.0] for i in range(len(rosbag_file_paths_red))])

img_paths.extend(rosbag_file_paths_none)
labels.extend([[0.0, 0.0, 0.0, 1.0] for i in range(len(rosbag_file_paths_none))])

### Getting Data from Heidelberg set

In [ ]:
HEIDELBERG_DATA_DIR = os.path.join('data', 'heidelberg')
HEIDELBERG_TRAIN_YAML_FILE = os.path.join(HEIDELBERG_DATA_DIR, 'train.yaml')

dataset_info = []

for yaml_path in [HEIDELBERG_TRAIN_YAML_FILE]:

    yaml_file = open(yaml_path, 'r')
    loaded_yaml = yaml_file.read()
    yaml_file.close()
    dataset_info.extend(yaml.load(loaded_yaml))

print('Heidelberg data set loaded')

In [ ]:
count_green = 0
count_yellow = 0
count_red = 0
count_none = 0

for i in range(len(dataset_info)):
    img_desc = dataset_info[i]
    img_path = os.path.join(HEIDELBERG_DATA_DIR, img_desc['path'])

    boxes = img_desc['boxes']

    if boxes is None or len(boxes) < 1:
        img_paths.append(img_path)
        labels.append([0.0, 0.0, 0.0, 1.0])
        count_none += 1
    else:
        t = np.array([0, 0, 0])

        for box in boxes:
            label = box['label']
            if label == 'Green':
                t += [1, 0, 0]
            elif label == 'Yellow':
                t += [0, 1, 0]
            elif label == 'Red':
                t += [0, 0, 1]

        t = t > 0
        if np.sum(t) <= 1:
            if t[0]:
                img_paths.append(img_path)
                labels.append([1.0, 0.0, 0.0, 0.0])
                count_green += 1
            elif t[1]:
                img_paths.append(img_path)
                labels.append([0.0, 1.0, 0.0, 0.0])
                count_yellow += 1
            elif t[2]:
                img_paths.append(img_path)
                labels.append([0.0, 0.0, 1.0, 0.0])
                count_red += 1
            else:
                img_paths.append(img_path)
                labels.append([0.0, 0.0, 0.0, 1.0])
                count_none += 1

print('From Heidelberg set - Green: {}, Yellow: {}, Red: {}, None: {}'.format(
    count_green, count_yellow, count_red, count_none))

### Combined training and test (validation) sets

In [ ]:
img_paths, labels = shuffle(img_paths, labels)

print('Total - imgs: {}, labels: {}'.format(len(img_paths), len(labels)))


img_paths_train, img_paths_test, labels_train, labels_test = train_test_split(img_paths, labels, test_size=0.2)

print('Train - imgs: {}, labels: {}'.format(len(img_paths_train), len(labels_train)))
print('Test - imgs: {}, labels: {}'.format(len(img_paths_test), len(labels_test)))

### Testing training/validation sets

In [ ]:
i = (int)(random.random() * len(img_paths))
img_path = img_paths[i]
print('Image Path: {}'.format(img_path))

img = cv2.resize(cv2.imread(img_paths[i]), (224, 224))
label = labels[i]

print('Image Shape: {}'.format(img.shape))
print('Image Label: {}'.format(label))
print('Image Class: {}'.format(
    'green' if label[0] else ('yellow' if label[1] else ('red' if label[2] else 'none'))))
plt.imshow(img[:,:,::-1])

## Traffic-Light Classifier

### Convolutional Neural Network

In [ ]:
NO_CLASSES = 4

def classifier():
    inputs = Input(shape=(224, 224, 3), name="in_input")
    resnet = ResNet50(weights='imagenet', input_tensor=inputs)
    x = resnet.output

    x = Dropout(0.5, name="out_dropout_1")(x)
    x = Dense(100, activation='relu', name="out_dense_1")(x)
    x = Dropout(0.5, name="out_dropout_2")(x)
    x = Dense(NO_CLASSES, activation='softmax', name="out_dense_2")(x)

    model = Model(inputs=inputs, outputs=x)
    return model

In [ ]:
# Uncoment to remove model and collect garbage
#del classifier_model
#gc.collect()

In [ ]:
classifier_model = classifier()
classifier_model.compile(loss="binary_crossentropy", optimizer='sgd', metrics=['accuracy'])
print(classifier_model.summary())

### Data sample generator

In [ ]:
def generate_sample(batch_size, img_paths_sample, labels_sample):
    while True:
        for batch_i in range(0, len(img_paths_sample), batch_size):
            imgs = []
            lbs = labels_sample[batch_i:batch_i+batch_size]
            for img_path in img_paths_sample[batch_i:batch_i+batch_size]:
                imgs.append(cv2.resize(cv2.imread(img_path), (224, 224)))
            yield np.array(imgs), np.array(lbs)

### Traning model

In [ ]:
BATCH_SIZE = 5
STEEPS_PER_EPOCH = len(img_paths_train) / BATCH_SIZE
TEST_STEEPS_PER_EPOCH = len(img_paths_test) / BATCH_SIZE

def train_classifier(epochs):

    classifier_model.fit_generator(
        generate_sample(BATCH_SIZE, img_paths_train, labels_train),
        steps_per_epoch=STEEPS_PER_EPOCH, 
        epochs=epochs,
        validation_data = generate_sample(BATCH_SIZE, img_paths_test, labels_test),
        validation_steps = TEST_STEEPS_PER_EPOCH)

In [ ]:
train_classifier(1)

### Testing the trained model

In [ ]:
i = (int)(random.random() * len(img_paths))
img_path = img_paths[i]
print('Image Path: {}'.format(img_path))

img = cv2.resize(cv2.imread(img_paths[i]), (224, 224))
label = labels[i]

print('Image Shape: {}'.format(img.shape))
print('Image Label: {}'.format(label))
print('Image Class: {}'.format(
    'green' if label[0] else ('yellow' if label[1] else ('red' if label[2] else 'none'))))
plt.imshow(img[:,:,::-1])


pred = classifier_model.predict(img.reshape(1,224,224,3))[0]
print('Prediction: {}'.format(pred))
img_cls = np.argmax(pred)
print('Predicted Class: {}'.format(
    'green' if img_cls==0 else ('yellow' if img_cls==1 else ('red' if img_cls==2 else 'none'))))

### Saving the trained model

In [ ]:
CLASSIFIER_MODEL_WEIGHTS_FILE = 'classifier_model_weights.h5'
CLASSIFIER_MODEL_YAML_FILE = 'classifier_model.yaml'

In [ ]:
# Saving the weights
classifier_model.save_weights(CLASSIFIER_MODEL_WEIGHTS_FILE)

# Saving the architecture
classifier_model_yaml = classifier_model.to_yaml()
with open(CLASSIFIER_MODEL_YAML_FILE, "w") as classifier_yaml_file:
    classifier_yaml_file.write(classifier_model_yaml)